# 第六章：機器學習 (MLlib)

Apache Spark MLlib 是 Spark 的機器學習函式庫，提供了分散式機器學習演算法和工具。

## 學習目標
- 了解 Spark MLlib 的基本概念
- 學習資料預處理和特徵工程
- 掌握分類、迴歸和聚類演算法
- 理解模型訓練和評估流程
- 學習模型管線 (Pipeline) 的使用

In [ ]:
# 初始化 Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 機器學習相關模組
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.clustering import KMeans, GaussianMixture
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# 建立 SparkSession
spark = SparkSession.builder \
    .appName("Spark MLlib Tutorial") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print(f"Spark 版本: {spark.version}")
print(f"可用核心數: {spark.sparkContext.defaultParallelism}")

## 1. 資料準備

我們將使用經典的鳶尾花資料集和房價資料集來示範不同的機器學習任務。

In [ ]:
# 建立鳶尾花資料集
from sklearn.datasets import load_iris
iris = load_iris()

# 轉換為 Spark DataFrame
iris_pandas = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_pandas['target'] = iris.target
iris_pandas['species'] = iris_pandas['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

iris_df = spark.createDataFrame(iris_pandas)
iris_df.show(10)
iris_df.printSchema()

In [ ]:
# 建立房價資料集（回歸問題）
from sklearn.datasets import load_boston
boston = load_boston()

boston_pandas = pd.DataFrame(boston.data, columns=boston.feature_names)
boston_pandas['price'] = boston.target

boston_df = spark.createDataFrame(boston_pandas)
boston_df.show(10)
boston_df.printSchema()

## 2. 資料探索和視覺化

In [ ]:
# 鳶尾花資料集統計
print("鳶尾花資料集統計:")
iris_df.describe().show()

# 類別分佈
print("\n類別分佈:")
iris_df.groupBy('species').count().show()

# 特徵相關性
print("\n特徵相關性:")
iris_stats = iris_df.select([col for col in iris_df.columns if col not in ['target', 'species']]).toPandas()
correlation_matrix = iris_stats.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('鳶尾花特徵相關性')
plt.show()

## 3. 特徵工程

在機器學習中，特徵工程是將原始資料轉換為適合模型訓練的特徵的過程。

In [ ]:
# 為分類問題準備特徵
def prepare_classification_features(df, feature_cols, label_col):
    """
    準備分類問題的特徵
    """
    # 1. 將標籤轉換為數值
    indexer = StringIndexer(inputCol=label_col, outputCol="label")
    
    # 2. 組合特徵向量
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    
    # 3. 標準化特徵
    scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
    
    # 建立預處理管線
    pipeline = Pipeline(stages=[indexer, assembler, scaler])
    
    # 訓練並轉換資料
    model = pipeline.fit(df)
    transformed_df = model.transform(df)
    
    return transformed_df, model

# 準備鳶尾花資料
feature_cols = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
iris_prepared, iris_preprocessor = prepare_classification_features(iris_df, feature_cols, 'species')

iris_prepared.select('species', 'label', 'features', 'scaledFeatures').show(5, truncate=False)

## 4. 分類問題

我們將使用多種分類演算法來預測鳶尾花的品種。

In [ ]:
# 分割訓練和測試資料
train_df, test_df = iris_prepared.randomSplit([0.8, 0.2], seed=42)

print(f"訓練資料數量: {train_df.count()}")
print(f"測試資料數量: {test_df.count()}")

# 檢查類別分佈
print("\n訓練集類別分佈:")
train_df.groupBy('species').count().show()

print("\n測試集類別分佈:")
test_df.groupBy('species').count().show()

### 4.1 邏輯迴歸

In [ ]:
# 邏輯迴歸模型
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="label", maxIter=10)
lr_model = lr.fit(train_df)

# 預測
lr_predictions = lr_model.transform(test_df)
lr_predictions.select('species', 'label', 'prediction', 'probability').show(10, truncate=False)

# 評估
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lr_predictions)
print(f"邏輯迴歸準確率: {lr_accuracy:.4f}")

# 其他評估指標
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
lr_f1 = evaluator_f1.evaluate(lr_predictions)
print(f"邏輯迴歸 F1 分數: {lr_f1:.4f}")

### 4.2 決策樹

In [ ]:
# 決策樹模型
dt = DecisionTreeClassifier(featuresCol="scaledFeatures", labelCol="label", maxDepth=5)
dt_model = dt.fit(train_df)

# 預測
dt_predictions = dt_model.transform(test_df)

# 評估
dt_accuracy = evaluator.evaluate(dt_predictions)
dt_f1 = evaluator_f1.evaluate(dt_predictions)
print(f"決策樹準確率: {dt_accuracy:.4f}")
print(f"決策樹 F1 分數: {dt_f1:.4f}")

# 查看特徵重要性
print("\n特徵重要性:")
feature_importance = dt_model.featureImportances
for i, importance in enumerate(feature_importance):
    print(f"{feature_cols[i]}: {importance:.4f}")

### 4.3 隨機森林

In [ ]:
# 隨機森林模型
rf = RandomForestClassifier(featuresCol="scaledFeatures", labelCol="label", numTrees=10, maxDepth=5)
rf_model = rf.fit(train_df)

# 預測
rf_predictions = rf_model.transform(test_df)

# 評估
rf_accuracy = evaluator.evaluate(rf_predictions)
rf_f1 = evaluator_f1.evaluate(rf_predictions)
print(f"隨機森林準確率: {rf_accuracy:.4f}")
print(f"隨機森林 F1 分數: {rf_f1:.4f}")

# 查看特徵重要性
print("\n特徵重要性:")
feature_importance = rf_model.featureImportances
for i, importance in enumerate(feature_importance):
    print(f"{feature_cols[i]}: {importance:.4f}")

## 5. 迴歸問題

使用房價資料集來示範迴歸問題的解決方案。

In [ ]:
# 準備回歸資料
regression_feature_cols = [col for col in boston_df.columns if col != 'price']

# 組合特徵向量
assembler = VectorAssembler(inputCols=regression_feature_cols, outputCol="features")
boston_assembled = assembler.transform(boston_df)

# 標準化特徵
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(boston_assembled)
boston_prepared = scaler_model.transform(boston_assembled)

# 分割資料
train_reg, test_reg = boston_prepared.randomSplit([0.8, 0.2], seed=42)

print(f"回歸訓練資料數量: {train_reg.count()}")
print(f"回歸測試資料數量: {test_reg.count()}")

# 查看目標變數統計
boston_prepared.describe('price').show()

### 5.1 線性迴歸

In [ ]:
# 線性迴歸模型
lr_reg = LinearRegression(featuresCol="scaledFeatures", labelCol="price", maxIter=10)
lr_reg_model = lr_reg.fit(train_reg)

# 預測
lr_reg_predictions = lr_reg_model.transform(test_reg)
lr_reg_predictions.select('price', 'prediction').show(10)

# 評估
reg_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

lr_rmse = reg_evaluator.setMetricName("rmse").evaluate(lr_reg_predictions)
lr_mae = reg_evaluator.setMetricName("mae").evaluate(lr_reg_predictions)
lr_r2 = reg_evaluator.setMetricName("r2").evaluate(lr_reg_predictions)

print(f"線性迴歸 RMSE: {lr_rmse:.4f}")
print(f"線性迴歸 MAE: {lr_mae:.4f}")
print(f"線性迴歸 R²: {lr_r2:.4f}")

# 模型係數
print(f"\n截距: {lr_reg_model.intercept:.4f}")
print("係數:")
coefficients = lr_reg_model.coefficients
for i, coef in enumerate(coefficients):
    print(f"{regression_feature_cols[i]}: {coef:.4f}")

### 5.2 隨機森林迴歸

In [ ]:
# 隨機森林迴歸模型
rf_reg = RandomForestRegressor(featuresCol="scaledFeatures", labelCol="price", numTrees=10, maxDepth=5)
rf_reg_model = rf_reg.fit(train_reg)

# 預測
rf_reg_predictions = rf_reg_model.transform(test_reg)

# 評估
rf_rmse = reg_evaluator.setMetricName("rmse").evaluate(rf_reg_predictions)
rf_mae = reg_evaluator.setMetricName("mae").evaluate(rf_reg_predictions)
rf_r2 = reg_evaluator.setMetricName("r2").evaluate(rf_reg_predictions)

print(f"隨機森林迴歸 RMSE: {rf_rmse:.4f}")
print(f"隨機森林迴歸 MAE: {rf_mae:.4f}")
print(f"隨機森林迴歸 R²: {rf_r2:.4f}")

# 特徵重要性
print("\n特徵重要性:")
feature_importance = rf_reg_model.featureImportances
importance_list = [(regression_feature_cols[i], float(importance)) for i, importance in enumerate(feature_importance)]
importance_list.sort(key=lambda x: x[1], reverse=True)

for feature, importance in importance_list[:10]:
    print(f"{feature}: {importance:.4f}")

## 6. 聚類分析

使用無監督學習進行聚類分析。

In [ ]:
# 使用鳶尾花資料進行聚類（不使用標籤）
clustering_data = iris_prepared.select('scaledFeatures')

# K-means 聚類
kmeans = KMeans(featuresCol="scaledFeatures", k=3, seed=42)
kmeans_model = kmeans.fit(clustering_data)

# 預測聚類
kmeans_predictions = kmeans_model.transform(iris_prepared)
kmeans_predictions.select('species', 'label', 'prediction').show(20)

# 計算聚類中心
centers = kmeans_model.clusterCenters()
print("聚類中心:")
for i, center in enumerate(centers):
    print(f"聚類 {i}: {center}")

# 計算 WSSSE (Within Set Sum of Squared Errors)
wssse = kmeans_model.summary.trainingCost
print(f"\nWSSE: {wssse:.4f}")

In [ ]:
# 比較聚類結果與真實標籤
clustering_comparison = kmeans_predictions.groupBy('species', 'prediction').count().orderBy('species', 'prediction')
clustering_comparison.show()

# 使用混淆矩陣視覺化
confusion_data = clustering_comparison.toPandas()
confusion_matrix = confusion_data.pivot(index='species', columns='prediction', values='count').fillna(0)

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt='g', cmap='Blues')
plt.title('K-means 聚類結果 vs 真實標籤')
plt.xlabel('聚類標籤')
plt.ylabel('真實標籤')
plt.show()

## 7. 模型管線 (Pipeline)

使用 Pipeline 將預處理和模型訓練步驟整合在一起。

In [ ]:
# 建立完整的機器學習管線
def create_classification_pipeline():
    """
    建立完整的分類管線
    """
    # 1. 標籤編碼
    indexer = StringIndexer(inputCol="species", outputCol="label")
    
    # 2. 特徵組合
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    
    # 3. 特徵標準化
    scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
    
    # 4. 分類模型
    rf = RandomForestClassifier(featuresCol="scaledFeatures", labelCol="label", numTrees=10)
    
    # 建立管線
    pipeline = Pipeline(stages=[indexer, assembler, scaler, rf])
    
    return pipeline

# 使用原始資料（未預處理）
train_raw, test_raw = iris_df.randomSplit([0.8, 0.2], seed=42)

# 建立和訓練管線
pipeline = create_classification_pipeline()
pipeline_model = pipeline.fit(train_raw)

# 預測
pipeline_predictions = pipeline_model.transform(test_raw)
pipeline_predictions.select('species', 'label', 'prediction', 'probability').show(10, truncate=False)

# 評估
pipeline_accuracy = evaluator.evaluate(pipeline_predictions)
print(f"管線模型準確率: {pipeline_accuracy:.4f}")

## 8. 模型調優

使用交叉驗證和參數網格搜索進行模型調優。

In [ ]:
# 參數網格搜索
def hyperparameter_tuning():
    """
    使用交叉驗證進行超參數調優
    """
    # 建立基本管線
    indexer = StringIndexer(inputCol="species", outputCol="label")
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
    rf = RandomForestClassifier(featuresCol="scaledFeatures", labelCol="label")
    
    pipeline = Pipeline(stages=[indexer, assembler, scaler, rf])
    
    # 定義參數網格
    paramGrid = ParamGridBuilder() \
        .addGrid(rf.numTrees, [5, 10, 20]) \
        .addGrid(rf.maxDepth, [3, 5, 7]) \
        .build()
    
    # 交叉驗證
    crossval = CrossValidator(estimator=pipeline,
                             estimatorParamMaps=paramGrid,
                             evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                             numFolds=3)
    
    # 訓練模型
    cv_model = crossval.fit(train_raw)
    
    return cv_model

# 執行超參數調優
print("開始超參數調優...")
cv_model = hyperparameter_tuning()

# 使用最佳模型進行預測
best_predictions = cv_model.transform(test_raw)
best_accuracy = evaluator.evaluate(best_predictions)
print(f"最佳模型準確率: {best_accuracy:.4f}")

# 取得最佳參數
best_model = cv_model.bestModel
best_rf = best_model.stages[-1]  # 最後一個階段是隨機森林
print(f"最佳參數:")
print(f"  numTrees: {best_rf.getNumTrees()}")
print(f"  maxDepth: {best_rf.getMaxDepth()}")

## 9. 模型持久化

保存和載入訓練好的模型。

In [ ]:
# 保存模型
model_path = "/tmp/spark_ml_model"

try:
    # 保存最佳模型
    cv_model.bestModel.write().overwrite().save(model_path)
    print(f"模型已保存至: {model_path}")
    
    # 載入模型
    from pyspark.ml import PipelineModel
    loaded_model = PipelineModel.load(model_path)
    
    # 使用載入的模型進行預測
    loaded_predictions = loaded_model.transform(test_raw)
    loaded_accuracy = evaluator.evaluate(loaded_predictions)
    print(f"載入模型準確率: {loaded_accuracy:.4f}")
    
except Exception as e:
    print(f"模型保存/載入錯誤: {e}")

## 10. 模型解釋性

理解模型的決策過程。

In [ ]:
# 特徵重要性分析
def analyze_feature_importance(model, feature_names):
    """
    分析特徵重要性
    """
    # 取得隨機森林模型
    rf_model = model.stages[-1]
    
    # 特徵重要性
    importances = rf_model.featureImportances
    
    # 建立特徵重要性 DataFrame
    importance_data = [(feature_names[i], float(importances[i])) for i in range(len(feature_names))]
    importance_df = spark.createDataFrame(importance_data, ['feature', 'importance'])
    
    # 排序並顯示
    importance_df.orderBy(col('importance').desc()).show()
    
    # 視覺化
    importance_pandas = importance_df.toPandas().sort_values('importance', ascending=True)
    
    plt.figure(figsize=(10, 6))
    plt.barh(importance_pandas['feature'], importance_pandas['importance'])
    plt.xlabel('特徵重要性')
    plt.title('隨機森林特徵重要性')
    plt.tight_layout()
    plt.show()
    
    return importance_df

# 分析最佳模型的特徵重要性
importance_df = analyze_feature_importance(cv_model.bestModel, feature_cols)

## 11. 模型性能比較

比較不同模型的性能。

In [ ]:
# 比較所有模型的性能
models_performance = {
    '邏輯迴歸': lr_accuracy,
    '決策樹': dt_accuracy,
    '隨機森林': rf_accuracy,
    '調優後隨機森林': best_accuracy
}

print("模型性能比較:")
for model_name, accuracy in models_performance.items():
    print(f"{model_name}: {accuracy:.4f}")

# 視覺化比較
model_names = list(models_performance.keys())
accuracies = list(models_performance.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(model_names, accuracies, color=['blue', 'green', 'red', 'purple'])
plt.ylabel('準確率')
plt.title('不同模型性能比較')
plt.ylim(0, 1.1)

# 在柱狀圖上顯示數值
for bar, accuracy in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{accuracy:.4f}', ha='center', va='bottom')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 12. 總結

在這個章節中，我們學習了：

1. **MLlib 基礎概念**：了解 Spark 機器學習庫的核心組件
2. **資料預處理**：特徵工程、標準化、編碼等技術
3. **分類問題**：邏輯迴歸、決策樹、隨機森林等演算法
4. **迴歸問題**：線性迴歸、隨機森林迴歸等方法
5. **聚類分析**：K-means 等無監督學習方法
6. **模型管線**：Pipeline 的使用和好處
7. **模型調優**：交叉驗證和參數網格搜索
8. **模型持久化**：保存和載入訓練好的模型
9. **模型解釋性**：特徵重要性分析
10. **性能比較**：不同模型的評估和比較

### 關鍵要點：
- 使用 Pipeline 可以將預處理和模型訓練步驟整合
- 交叉驗證是避免過度擬合的重要技術
- 特徵工程對模型性能有重大影響
- 模型解釋性對於理解業務問題很重要
- 不同演算法適合不同類型的問題

### 下一步：
- 學習更高級的特徵工程技術
- 探索深度學習和神經網絡
- 了解大規模機器學習的最佳實踐
- 學習模型部署和監控

In [ ]:
# 清理資源
spark.stop()
print("Spark 會話已結束")